In [4]:
import math
import random
import numpy as np
import pandas as pd
from datetime import datetime


import tensorflow as tf
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()


from tqdm import tqdm_notebook, tqdm
from collections import deque

In [5]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    # Define hyperparamaters
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
        
    # Call a function  to build a model trought this class constructor
    # More parameters could be ustilized to programaticaly define network size (layers and neurons)
    self.model = self.model_builder()
    
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))    
    return model
  
  
  
  
  # Trade function that takes state as an input and returns an action 
  # to perform in perticular state 
  def trade(self, state):
    
    # Should we perform a renadom generated action or action defined in model?
    
    # If value from our random generator is smaller or equal to our epsilon 
    #     then we will retun a random action from action_space [0-3)
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    # If our random is greater than epsilon then we will use model to perform action
    actions = self.model.predict(state)
    # return only a one number defining an action (#Stay - 0 , Buy - 1, Sell - 2) 
    #    that has maximum probability
    return np.argmax(actions[0])
  
  
  
  def batch_train(self, batch_size):
    
    batch = []
    
    # Iterrate in momory, we do not want to randolmy select data as we are dealing with 
    #    time constraint data. We will always sample from the end of memory size of bath
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      # insert data from memory to batch      
      batch.append(self.memory[i])
    
    
    # Iterate trought batch of data and train the model for each sample from batch
    # Order of variables in for loop is important
    for state, action, reward, next_state, done in batch:
      # Reward if agent is in terminal state
      reward = reward
      # Check that agent is not in terminal state
      # If not in terminal state calculate reward for actions that could be played
      if not done:
        # Discounted total reward:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])        
      # Target variable that is predicted by the model (action)
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    # We will decrease epsilon parameter that is 1 as defined in __init__  so
    #    so we can stop performing random actions at some point
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [65]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

  
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))


def dataset_loader(stock_name):
  
  #Use pandas data reader for reading stock data from warious sources like "yahoo", "google"
  dataset = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2024-01-01")
   
  # Get start and end time to variables from dataset
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  # Model will use "Close" column for training 
  close = dataset['Close']

  return close

# Data -> dataset to predict from, gathered by data:loader()
# Timestep -> Day in the dataset that we want to predict for [0:datalength]
# window_suze -> how many days in past we want to use to predict current status[1:datalength]
#         Try different setup to see what creates best fit
def state_creator(data, timestep, window_size):
  
  #starting day of our state
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data.iloc[starting_id:timestep+1]
  else:
    #Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data.iloc[0]] + list(data[0:timestep+1])
    
  state = []
  #Iterate trough whole windowed_data minus current state (-1)
  for i in range(window_size - 1):
    #Normalize the difference from current day and the next day
    #Because the prices can be very different and we want them on same scale
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])


In [56]:
# Tage data for Apple
stock_name = "AAPL"
data = dataset_loader(stock_name)

[*********************100%%**********************]  1 of 1 completed


In [57]:
timestep = 0
window_size = 10

starting_id = timestep - window_size + 1
  
if starting_id >= 0:
    windowed_data = data.iloc[starting_id:timestep+1]
else:
    #Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data.iloc[0]] + list(data[0:timestep+1])

In [78]:
window_size = 10
episodes = 1000 # same as epoch

batch_size = 32
data_samples = len(data) - 1 # discard last value, that we will predict on

In [41]:
data_samples

1715

In [37]:
trader = AI_Trader(window_size)
trader.model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f89f3abfd90>>

In [90]:
timestep = 15
window_size = 10
  
#starting day of our state
starting_id = timestep - window_size + 1
  
if starting_id >= 0:
   windowed_data = data.iloc[starting_id:timestep+1]
else:
    #Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data.iloc[0]] + list(data[0:timestep+1])
    
state = []
#Iterate trough whole windowed_data minus current state (-1)
for i in range(window_size - 1):
    #Normalize the difference from current day and the next day
    #Because the prices can be very different and we want them on same scale
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
np.array([state])

/tmp/ipykernel_8794/2614383990.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))


array([[0.36123513, 0.62714877, 0.31002487, 0.62245933, 0.30153556,
        0.69635532, 0.35663219, 0.81000031, 0.87761124]])

In [86]:
sigmoid(windowed_data[8+1] - windowed_data[8])

0.5

In [83]:
windowed_data

[225.24000549316406,
 225.24000549316406,
 225.24000549316406,
 225.24000549316406,
 225.24000549316406,
 225.24000549316406,
 225.24000549316406,
 225.24000549316406,
 225.24000549316406,
 225.24000549316406]

In [84]:
data

Date
2017-01-03    225.240005
2017-01-04    226.580002
2017-01-05    226.399994
2017-01-06    227.210007
2017-01-09    226.460007
                 ...    
2023-10-20    421.190002
2023-10-23    420.459991
2023-10-24    423.630005
2023-10-25    417.549988
2023-10-26    412.549988
Name: Close, Length: 1716, dtype: float64

In [79]:
episode = 1
state = state_creator(data, 0 , window_size + 1)
state

array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]])

In [66]:
episode = 1
state = state_creator(data, 0 , window_size + 1)
total_profit = 0

trader.inventory = []

for t in tqdm(range(1)):
    action = trader.trade(state)

        # Use action to get to next state(t+)
    next_state = state_creator(data, t+1, window_size + 1)
    # As we did not calculate anything up to this point reward is 0
    reward = 0
    
    if action == 1: #Buying
      # Put buyed stock to inventory to trade with
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    # To sell we need to have something in inventory  
    elif action == 2 and len(trader.inventory) > 0: #Selling
      # Check buy price, pop removes first value from list
      buy_price = trader.inventory.pop(0)
      
      # If we gain money (current price - buy price) we have reward 
      #    if we lost money then reward is 0
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    # if t is last sample in our dateset we are done
    #     we do not have any steps to perform in current episode
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    # Append all data to trader-agent memory, experience buffer
    trader.memory.append((state, action, reward, next_state, done))
    
    # change state to next state, so we are done with an episode
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    # Chekc if we have more information in our memory than batch size
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
  
  # Save the model every 10 episodes
    if episode % 10 == 0:
       trader.model.save("ai_trader_{}.h5".format(episode))


  0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 57ms/step


100%|██████████| 1/1 [00:22<00:00, 22.60s/it]


In [77]:
trader.memory

deque([(array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]),
        2,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        0,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.79248934, 0.45511914]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.79248934, 0.45511914]]),
        1,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.79248934, 0.45511914, 0.69211224]]),
        False),
       (array([[0.5       , 0.5       , 0.

In [75]:
batch_size

32

In [68]:
trader.memory[0]

(array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]),
 2,
 0,
 array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
 False)

In [74]:
data

Date
2017-01-03    225.240005
2017-01-04    226.580002
2017-01-05    226.399994
2017-01-06    227.210007
2017-01-09    226.460007
                 ...    
2023-10-20    421.190002
2023-10-23    420.459991
2023-10-24    423.630005
2023-10-25    417.549988
2023-10-26    412.549988
Name: Close, Length: 1716, dtype: float64

In [69]:
state

array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]])

In [71]:
trader.model.predict(state)

1/1 [==============================] - 0s 230ms/step


array([[45.306393, 43.670013, 46.8499  ]], dtype=float32)

In [89]:
state

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

In [88]:
trader.trade(state)

2

In [72]:
trader.memory

deque([(array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]),
        2,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        0,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.79248934, 0.45511914]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.79248934, 0.45511914]]),
        1,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.79248934, 0.45511914, 0.69211224]]),
        False),
       (array([[0.5       , 0.5       , 0.